In [399]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import googlemaps
from datetime import datetime
import requests
import seaborn as sns
from sklearn.impute import SimpleImputer
from matplotlib.ticker import FixedLocator

In [270]:
# Carga del archivo
lesiones = pd.read_excel('../datasets/raw/lesiones.xlsx', sheet_name='HECHOS', na_values=['SD', 'sd'])
lesiones

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,longitud,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9.0,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-5.840891e+01,-3.455966e+01,CICLISTA,NaN,CICLISTA-SD,NaN,NaN,NaN,NaN,x,NaN
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1.0,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-5.844351e+01,-3.466913e+01,AUTO,NaN,AUTO-SD,NaN,x,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2.0,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-5.846834e+01,-3.467756e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2.0,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-5.843742e+01,-3.464735e+01,PEATON,NaN,PEATON-SD,x,NaN,NaN,NaN,NaN,NaN
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4.0,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-5.839822e+01,-3.460458e+01,AUTO,NaN,AUTO-SD,NaN,NaN,x,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1,2021,12,31,2021-12-31 00:00:00,19:30:00,19.0,"ALBERDI, JUAN BAUTISTA AV. y GUAMINI",9,AVENIDA,NaN,"ALBERDI, JUAN BAUTISTA AV.",0.0,GUAMINI,Point (95400.5441905024 96621.6007711877),-5.851348e+01,-3.465971e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23781,LC-2021-0652865,2,2021,12,31,2021-12-31 00:00:00,19:40:00,19.0,"ALBERDI, JUAN BAUTISTA AV. 4436",9,AVENIDA,NaN,"ALBERDI, JUAN BAUTISTA AV.",4436.0,NaN,Point (97705.4491469278 98614.8825470092),-5.848833e+01,-3.464175e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23782,LC-2021-0652907,1,2021,12,31,2021-12-31 00:00:00,20:00:00,20.0,NaN,1,NaN,Barrio Padre Carlos Mugica (Villa 31 bis),NaN,NaN,NaN,Point (107377.10694920316745993 105120.7873965...,-5.838289e+01,-3.458308e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23783,LC-2021-0652921,1,2021,12,31,2021-12-31 00:00:00,22:00:00,22.0,LINIERS VIRREY y MORENO,5,CALLE,NaN,LINIERS VIRREY,0.0,MORENO,Point (104472.702045632 101660.930364258),-5.841453e+01,-3.461429e+01,MOTO,TRANSPORTE PUBLICO,MOTO-TRANSPORTE PUBLICO,x,0,x,0,0,GRAVE


Función que permetirá ir corroborando la integridad de nuestros datos

In [271]:
def resumen_tipos_de_datos(df):
    resumen = []
    total_filas = len(df)
    for col in df.columns:
        tipos_presentes = df[col].apply(lambda x: type(x).__name__).unique()
        cantidad_nulos = df[col].isnull().sum()
        porcentaje_nulos = round((cantidad_nulos / total_filas) * 100, 2)

        resumen.append({
            'Columna': col,
            'Tipos de Dato': tipos_presentes.tolist(),
            'Cantidad de Nulos': cantidad_nulos,
            '% de Nulos': porcentaje_nulos
        })
    return pd.DataFrame(resumen)


In [272]:
resumen_tipos_de_datos(lesiones)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,id,[str],0,0.00
1,n_victimas,[int],0,0.00
2,aaaa,[int],0,0.00
3,mm,[int],0,0.00
4,dd,[int],0,0.00
5,fecha,"[datetime, str]",0,0.00
6,hora,"[time, float, str]",9,0.04
7,franja_hora,[float],9,0.04
8,direccion_normalizada,"[float, str]",10868,45.69
9,comuna,"[int, str, float]",1015,4.27


### 1. Eliminación de columnas

La columna 'fecha' se conservará, las columnas `aaaa`, `mm`, y `dd`, serán eliminados por ser considerados datos redundates.

In [273]:
lesiones.drop(columns=['aaaa','mm','dd'], inplace=True)

Eliminamos columna `hora`, para conservar la columna `franja_hora`, porque tiene la hora en entero, esto optimiza futuros análisis.

In [274]:
lesiones.drop(columns=['hora'],inplace=True)

Para la ubicación se utilizará las columnas `latitud` y `longitud`. Procederemos a eliminar las columnas relacionadas a direcciones.

In [275]:
# lesiones.drop(columns=['otra_direccion','calle','altura','cruce','geocodificacion_CABA','direccion_normalizada'],inplace=True)

La columna `participantes`detalla la información sobre los involucrados en el incidente, por tanto, columnas para detallar la presencia de **moto**, **auto**, **transporte público**, **camión** y **ciclista** resultan redundantes. 

In [276]:
lesiones.drop(columns=['moto','auto','transporte_publico','camion','ciclista'],inplace=True)

In [277]:
# Corroboramos la integridad de nuestro DF
resumen_tipos_de_datos(lesiones)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,id,[str],0,0.00
1,n_victimas,[int],0,0.00
2,fecha,"[datetime, str]",0,0.00
3,franja_hora,[float],9,0.04
4,direccion_normalizada,"[float, str]",10868,45.69
5,comuna,"[int, str, float]",1015,4.27
6,tipo_calle,"[float, str]",11045,46.44
7,otra_direccion,"[str, float]",18295,76.92
8,calle,"[float, str]",10918,45.90
9,altura,[float],11014,46.31


La columna `gravedad` muestra una cantidad alarmante de valores nulos. No obstante, al revisar el diccionario, se aclara que se debe considerar como gravedad `leve`.

In [278]:
lesiones['gravedad'] = lesiones['gravedad'].fillna('leve')

In [279]:
# Verificamos como va quedando el DF
df_lesiones_hechos = lesiones
df_lesiones_hechos

,id,n_victimas,fecha,franja_hora,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,longitud,latutid,victima,acusado,participantes,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9.0,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-5.840891e+01,-3.455966e+01,CICLISTA,NaN,CICLISTA-SD,leve
1,LC-2019-0000053,1,2019-01-01 00:00:00,1.0,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-5.844351e+01,-3.466913e+01,AUTO,NaN,AUTO-SD,leve
2,LC-2019-0000063,1,2019-01-01 00:00:00,2.0,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-5.846834e+01,-3.467756e+01,NaN,NaN,SD-SD,leve
3,LC-2019-0000079,1,2019-01-01 00:00:00,2.0,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-5.843742e+01,-3.464735e+01,PEATON,NaN,PEATON-SD,leve
4,LC-2019-0000082,4,2019-01-01 00:00:00,4.0,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-5.839822e+01,-3.460458e+01,AUTO,NaN,AUTO-SD,leve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1,2021-12-31 00:00:00,19.0,"ALBERDI, JUAN BAUTISTA AV. y GUAMINI",9,AVENIDA,NaN,"ALBERDI, JUAN BAUTISTA AV.",0.0,GUAMINI,Point (95400.5441905024 96621.6007711877),-5.851348e+01,-3.465971e+01,NaN,NaN,SD-SD,leve
23781,LC-2021-0652865,2,2021-12-31 00:00:00,19.0,"ALBERDI, JUAN BAUTISTA AV. 4436",9,AVENIDA,NaN,"ALBERDI, JUAN BAUTISTA AV.",4436.0,NaN,Point (97705.4491469278 98614.8825470092),-5.848833e+01,-3.464175e+01,NaN,NaN,SD-SD,leve
23782,LC-2021-0652907,1,2021-12-31 00:00:00,20.0,NaN,1,NaN,Barrio Padre Carlos Mugica (Villa 31 bis),NaN,NaN,NaN,Point (107377.10694920316745993 105120.7873965...,-5.838289e+01,-3.458308e+01,NaN,NaN,SD-SD,leve
23783,LC-2021-0652921,1,2021-12-31 00:00:00,22.0,LINIERS VIRREY y MORENO,5,CALLE,NaN,LINIERS VIRREY,0.0,MORENO,Point (104472.702045632 101660.930364258),-5.841453e+01,-3.461429e+01,MOTO,TRANSPORTE PUBLICO,MOTO-TRANSPORTE PUBLICO,GRAVE


En la columna `comuna`, los datos 'No Especificada' serán convertidos a nulos. 

In [280]:
df_lesiones_hechos['comuna'] = df_lesiones_hechos['comuna'].replace('No Especificada', np.NaN)

### 2. Cambio de tipos de datos
Las columnas `franja_hora`, `tipo_calle`, `comuna` y ``acusado`` serán transformadas a tipo categórico.

In [281]:
# Normalizamos todos los valores nulos
mask = df_lesiones_hechos.applymap(lambda x: x == '<NA>')
df_lesiones_hechos[mask] = np.NaN

In [282]:
df_lesiones_hechos['tipo_calle'] = df_lesiones_hechos['tipo_calle'].astype('category')
df_lesiones_hechos['acusado'] = df_lesiones_hechos['acusado'].astype('category')

La columna ``fecha`` la convertiremos a formato fecha

In [283]:
df_lesiones_hechos['fecha'] = pd.to_datetime(df_lesiones_hechos['fecha'], errors='coerce')

La columna ``comuna`` y ``franja_hora`` la convertiremos a formato INT

In [284]:
df_lesiones_hechos['comuna'] = df_lesiones_hechos['comuna'].astype('Int64')
df_lesiones_hechos['franja_hora'] = df_lesiones_hechos['franja_hora'].astype('Int64')

In [285]:
df_lesiones_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     23785 non-null  object        
 1   n_victimas             23785 non-null  int64         
 2   fecha                  23785 non-null  datetime64[ns]
 3   franja_hora            23776 non-null  Int64         
 4   direccion_normalizada  12917 non-null  object        
 5   comuna                 22328 non-null  Int64         
 6   tipo_calle             12740 non-null  category      
 7   otra_direccion         5490 non-null   object        
 8   calle                  12867 non-null  object        
 9   altura                 12771 non-null  float64       
 10  cruce                  9407 non-null   object        
 11  geocodificacion_CABA   22533 non-null  object        
 12  longitud               22314 non-null  float64       
 13  l

### 3. Renombrar columnas

In [286]:
df_lesiones_hechos.head()

,id,n_victimas,fecha,franja_hora,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,longitud,latutid,victima,acusado,participantes,gravedad
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-58.408911,-34.559658,CICLISTA,NaN,CICLISTA-SD,leve
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-58.443510,-34.669125,AUTO,NaN,AUTO-SD,leve
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-58.468335,-34.677556,NaN,NaN,SD-SD,leve
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-58.437425,-34.647349,PEATON,NaN,PEATON-SD,leve
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-58.398225,-34.604579,AUTO,NaN,AUTO-SD,leve


Cambiamos todos los nombres de las columnas, para poder darle un mejor seguimiento

In [287]:
df_lesiones_hechos = df_lesiones_hechos.rename(columns={'id':'id_hecho'})
df_lesiones_hechos = df_lesiones_hechos.rename(columns={'franja_hora':'hh'})
df_lesiones_hechos = df_lesiones_hechos.rename(columns={'latutid':'latitud'})

In [288]:
df_lesiones_hechos.head()

,id_hecho,n_victimas,fecha,hh,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,longitud,latitud,victima,acusado,participantes,gravedad
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-58.408911,-34.559658,CICLISTA,NaN,CICLISTA-SD,leve
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-58.443510,-34.669125,AUTO,NaN,AUTO-SD,leve
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-58.468335,-34.677556,NaN,NaN,SD-SD,leve
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-58.437425,-34.647349,PEATON,NaN,PEATON-SD,leve
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-58.398225,-34.604579,AUTO,NaN,AUTO-SD,leve


In [289]:
# Transformar a minúsculas las columnas en df_lesiones_hechos
columnas_a_minusc = ['tipo_calle', 'gravedad', 'victima', 'acusado', 'participantes']
df_lesiones_hechos[columnas_a_minusc] = df_lesiones_hechos[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

# Correciones ortográficas
mapeo_correcciones_victima_lesiones = {'peaton': 'peatón','transporte publico': 'transporte público','movil': 'móvil','otro': 'otro',
'camion': 'camión','monopatin': 'monopatín','utilitario': 'utilitario'}
df_lesiones_hechos['victima'] = df_lesiones_hechos['victima'].replace(mapeo_correcciones_victima_lesiones)

In [290]:
df_lesiones_hechos.head()

,id_hecho,n_victimas,fecha,hh,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,longitud,latitud,victima,acusado,participantes,gravedad
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,nan,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-58.408911,-34.559658,ciclista,nan,ciclista-sd,leve
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,nan,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-58.443510,-34.669125,auto,nan,auto-sd,leve
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,nan,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-58.468335,-34.677556,nan,nan,sd-sd,leve
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,nan,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-58.437425,-34.647349,peatón,nan,peaton-sd,leve
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,nan,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-58.398225,-34.604579,auto,nan,auto-sd,leve


In [291]:
df_lesiones_hechos.replace('nan', np.nan, inplace=True)

In [292]:
df_lesiones_hechos.head()

,id_hecho,n_victimas,fecha,hh,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,longitud,latitud,victima,acusado,participantes,gravedad
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-58.408911,-34.559658,ciclista,NaN,ciclista-sd,leve
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-58.443510,-34.669125,auto,NaN,auto-sd,leve
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-58.468335,-34.677556,NaN,NaN,sd-sd,leve
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-58.437425,-34.647349,peatón,NaN,peaton-sd,leve
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-58.398225,-34.604579,auto,NaN,auto-sd,leve


# ETL dataframe ``LESIONES-VICTIMAS``

In [293]:
# Carga del archivo
df_lesiones_victimas = pd.read_excel('../datasets/raw/lesiones.xlsx', sheet_name='VICTIMAS', na_values=['SD', 'sd'])
df_lesiones_victimas

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,Varon,57.0,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,Varon,NaN,NaN
3,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Varon,45.0,NaN
4,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Mujer,45.0,NaN
...,...,...,...,...,...,...,...,...,...
27600,LC-2021-0451911,2021,9,11,2021-09-11,TRANSPORTE PUBLICO,Varon,87.0,NaN
27601,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,Mujer,60.0,NaN
27602,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,Mujer,32.0,NaN
27603,LC-2021-0201378,2021,5,2,2021-05-02,MOTO,Varon,32.0,NaN


La columna ``FECHA`` será conservada, las columnas **AAA**, **DD**, y **MM** serán eliminadas, por ser información redundate.

In [294]:
df_lesiones_victimas.drop(columns=['AAA','MM','DD'],inplace=True)

In [295]:
resumen_tipos_de_datos(df_lesiones_victimas)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,ID hecho,[str],0,0.00
1,FECHA,[Timestamp],0,0.00
2,VEHICULO_VICTIMA,"[float, str]",16272,58.95
3,SEXO,"[str, float]",2228,8.07
4,EDAD_VICTIMA,[float],4471,16.20
5,GRAVEDAD,"[float, str]",27068,98.05


La columna `gravedad` muestra una cantidad alarmante de valores nulos. No obstante, al revisar el diccionario, se aclara que se debe considerar como gravedad `leve`.

In [296]:
df_lesiones_victimas['GRAVEDAD'] = df_lesiones_victimas['GRAVEDAD'].fillna('leve')

### 1. Cambio de tipo de datos

Conversión columnas ``VEHICULO_VICTIMA``, ``GRAVEDAD`` y ``SEXO`` a tipo categórico

In [297]:
# Cambio a tipo categórico
df_lesiones_victimas['VEHICULO_VICTIMA'] = df_lesiones_victimas['VEHICULO_VICTIMA'].astype('category')
df_lesiones_victimas['GRAVEDAD'] = df_lesiones_victimas['GRAVEDAD'].astype('category')
df_lesiones_victimas['SEXO'] = df_lesiones_victimas['SEXO'].astype('category')

Conversión columna `EDAD_VICTIMA` a tipo INT, para facilitar su posterior análisis

In [298]:
df_lesiones_victimas['EDAD_VICTIMA'] = df_lesiones_victimas['EDAD_VICTIMA'].astype('Int64')

In [299]:
df_lesiones_victimas.head()

,ID hecho,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019-01-01,NaN,Varon,57,leve
1,LC-2019-0000063,2019-01-01,NaN,NaN,<NA>,leve
2,LC-2019-0000079,2019-01-01,NaN,Varon,<NA>,leve
3,LC-2019-0000082,2019-01-01,NaN,Varon,45,leve
4,LC-2019-0000082,2019-01-01,NaN,Mujer,45,leve


### 2. Renombrar Columnas
Renombrar columnas del DF ``df_lesiones_victimas`` para la concordancia con el DF ``df_lesiones_hechos``

In [300]:
# Convertimos todo a minúsculas
df_lesiones_victimas.columns = df_lesiones_victimas.columns.str.lower()
df_lesiones_victimas.rename(columns={'id hecho': 'id_hecho', 'fecha ': 'fecha', 'sexo': 'sexo_victima'}, inplace=True)

In [301]:
df_lesiones_victimas

,id_hecho,fecha,vehiculo_victima,sexo_victima,edad_victima,gravedad
0,LC-2019-0000053,2019-01-01,NaN,Varon,57,leve
1,LC-2019-0000063,2019-01-01,NaN,NaN,<NA>,leve
2,LC-2019-0000079,2019-01-01,NaN,Varon,<NA>,leve
3,LC-2019-0000082,2019-01-01,NaN,Varon,45,leve
4,LC-2019-0000082,2019-01-01,NaN,Mujer,45,leve
...,...,...,...,...,...,...
27600,LC-2021-0451911,2021-09-11,TRANSPORTE PUBLICO,Varon,87,leve
27601,LC-2021-0530228,2021-10-25,TRANSPORTE PUBLICO,Mujer,60,leve
27602,LC-2021-0530228,2021-10-25,TRANSPORTE PUBLICO,Mujer,32,leve
27603,LC-2021-0201378,2021-05-02,MOTO,Varon,32,leve


### 3. Renombrar datos de las columnas

In [302]:
df_lesiones_victimas['sexo_victima'].unique()

['Varon', NaN, 'Mujer', 'varon', 'mujer', 'MUJER', 'Mujer ']
Categories (6, object): ['MUJER', 'Mujer', 'Mujer ', 'Varon', 'mujer', 'varon']

In [303]:
# Mapeo de categorías
mapeo_sexo = {'Varon': 'Masculino', 'varon': 'Masculino', 'Mujer': 'Femenino', 'mujer': 'Femenino', 'MUJER': 'Femenino', 'Mujer ': 'Femenino'}
# Aplicar el mapeo a la columna 'sexo_victima'
df_lesiones_victimas['sexo_victima'] = df_lesiones_victimas['sexo_victima'].map(mapeo_sexo, na_action='ignore')

In [304]:
df_lesiones_victimas['sexo_victima'].unique()

array(['Masculino', nan, 'Femenino'], dtype=object)

Corregimos la ortografía de la columna ``vehiculo_victima``

In [305]:
# Diccionario dado:
diccionario_correcciones = {'movil': 'móvil','camion': 'camión','monopatin': 'monopatín','peaton':'peatón',
                            'transporte publico':'transporte público'}
# Correción acorde al diccionario
df_lesiones_victimas['vehiculo_victima'] = df_lesiones_victimas['vehiculo_victima'].replace(diccionario_correcciones.keys(), diccionario_correcciones.values(), regex=True)

In [306]:
# Convertimos los valores 'nan' en valores nulos
df_lesiones_victimas.replace('nan', np.nan, inplace=True)

In [307]:
df_lesiones_victimas.head()

,id_hecho,fecha,vehiculo_victima,sexo_victima,edad_victima,gravedad
0,LC-2019-0000053,2019-01-01,NaN,Masculino,57,leve
1,LC-2019-0000063,2019-01-01,NaN,NaN,<NA>,leve
2,LC-2019-0000079,2019-01-01,NaN,Masculino,<NA>,leve
3,LC-2019-0000082,2019-01-01,NaN,Masculino,45,leve
4,LC-2019-0000082,2019-01-01,NaN,Femenino,45,leve


In [308]:
# Creamos una función para asignar el rango etario

def asignar_rango_etario(edad):
    rangos_etarios = {
        (0, 17): "1_ 0 a 17 años",
        (18, 29): "2_ 18 a 29 años",
        (30, 39): "3_ 30 a 39 años",
        (40, 49): "4_ 40 a 49 años",
        (50, 59): "5_ 50 a 59 años",
        (60, float('inf')): "6_ 60 años o más"
    }

    for rango, etiqueta in rangos_etarios.items():
        if rango[0] <= edad <= rango[1]:
            return etiqueta

    return np.nan

In [309]:
# Aplicamos la funcion para asignar el rango etario
df_lesiones_victimas['RANGO_ETARIO'] = df_lesiones_victimas['edad_victima'].apply(lambda x: asignar_rango_etario(x))
df_lesiones_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_hecho          27605 non-null  object        
 1   fecha             27605 non-null  datetime64[ns]
 2   vehiculo_victima  11333 non-null  category      
 3   sexo_victima      25377 non-null  object        
 4   edad_victima      23134 non-null  Int64         
 5   gravedad          27605 non-null  category      
 6   RANGO_ETARIO      23134 non-null  object        
dtypes: Int64(1), category(2), datetime64[ns](1), object(3)
memory usage: 1.1+ MB


# ETL dataframe ``HOMICIDIOS-HECHOS``

In [310]:
homicidios_hechos= pd.read_excel('../datasets/raw/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])

In [311]:
homicidios_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


### 1. Eliminación de columnas
La columna `FECHA` será conservada, las columnas **AAAA**, **DD**, y **MM**, serán eliminadas por ser consideradas información redundate.

In [312]:
homicidios_hechos.drop(columns=['AAAA','DD','MM'],inplace=True)

La columna ``HH``, que representa la hora entera, será conservada, mientras que la columna **HORA**, que incluye los minutos, será eliminada.

In [313]:
homicidios_hechos.drop(columns=['HORA'],inplace=True)

Para la ubicación del hecho, se ha decidido utilizar las columnas `pos X` y `pos Y`, procedemos a eliminar las demas columnas que hacen referencia a ubicación.

In [314]:
homicidios_hechos.drop(columns=['LUGAR_DEL_HECHO','Calle','Altura','Cruce','Dirección Normalizada','XY (CABA)'],inplace=True)

### 2. Valores Nulos
Calcular el porcentaje de valores nulos por columnas

In [315]:
resumen_tipos_de_datos(homicidios_hechos)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,ID,[str],0,0.00
1,N_VICTIMAS,[int],0,0.00
2,FECHA,[Timestamp],0,0.00
3,HH,[float],1,0.14
4,TIPO_DE_CALLE,[str],0,0.00
5,COMUNA,[int],0,0.00
6,pos x,[str],0,0.00
7,pos y,[str],0,0.00
8,PARTICIPANTES,[str],0,0.00
9,VICTIMA,"[str, float]",9,1.29


El porcentaje de valores nulos en el conjunto de datos es bajo, afectando solo a 3 columnas. Sin embargo, se identificaron irregularidades en la columna de "Comuna", donde la presencia de "Comuna 0" indica una inconsistencia, ya que las comunas deben estar en el rango de 1 a 15. Estos casos también serán tratados como valores nulos. Además, se observaron puntos en las columnas "Pos X" y "Pos Y", y se decidirá reemplazarlos por valores nulos para mantener la consistencia en la representación de coordenadas geográficas.

In [316]:
# Reemplazo de la Comuna 0 por Valores Nulos
homicidios_hechos.loc[homicidios_hechos['COMUNA'] == 0, 'COMUNA'] = np.nan

# Reemplazo de los '.' por valores nulos de las columnas pos x y pos y
homicidios_hechos.loc[homicidios_hechos['pos x'] == '.' ,'pos x'] = np.nan
homicidios_hechos.loc[homicidios_hechos['pos y'] == '.' ,'pos y'] = np.nan

In [317]:
resumen_tipos_de_datos(homicidios_hechos)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,ID,[str],0,0.00
1,N_VICTIMAS,[int],0,0.00
2,FECHA,[Timestamp],0,0.00
3,HH,[float],1,0.14
4,TIPO_DE_CALLE,[str],0,0.00
5,COMUNA,[float],2,0.29
6,pos x,"[str, float]",12,1.72
7,pos y,"[str, float]",12,1.72
8,PARTICIPANTES,[str],0,0.00
9,VICTIMA,"[str, float]",9,1.29


### 3. Cambio de tipo de datos
Las columnas ``HH``, ``TIPO_DE_CALLE``, ``COMUNA`` y ``ACUSADO`` las pasaremos a tipo **CATEGORY**

In [318]:
homicidios_hechos['HH'] = homicidios_hechos['HH'].astype('category')
homicidios_hechos['TIPO_DE_CALLE'] = homicidios_hechos['TIPO_DE_CALLE'].astype('category')
homicidios_hechos['COMUNA'] = homicidios_hechos['COMUNA'].astype('category')
homicidios_hechos['ACUSADO'] = homicidios_hechos['ACUSADO'].astype('category')

Las columnas 'pos X' y 'pos Y' serán transformadas al tipo **FLOAT**

In [319]:
homicidios_hechos['pos x'] = homicidios_hechos['pos x'].astype('float')
homicidios_hechos['pos y'] = homicidios_hechos['pos y'].astype('float')

### 4. Renombrar columnas

In [320]:
# Pasamos a minúscilas todos los encabezados
homicidios_hechos.columns = homicidios_hechos.columns.str.lower()

Renombrar columnas: ``id``, ``pos x`` y ``pos y``

In [321]:
homicidios_hechos.rename(columns={'id': 'id_hecho', 'pos x': 'longitud', 'pos y': 'latitud','tipo_de_calle':'tipo_calle'}, inplace=True)

Pasar a minúsculas el contenido de las columnas `tipo_calle`, `participantes`, `victima` y `acusado`

In [322]:
columnas_a_minusc = ['tipo_calle', 'participantes', 'victima', 'acusado']
homicidios_hechos[columnas_a_minusc] = homicidios_hechos[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

In [323]:
homicidios_hechos.head()

,id_hecho,n_victimas,fecha,hh,tipo_calle,comuna,longitud,latitud,participantes,victima,acusado
0,2016-0001,1,2016-01-01,4.0,avenida,8.0,-58.475340,-34.687570,moto-auto,moto,auto
1,2016-0002,1,2016-01-02,1.0,gral paz,9.0,-58.508775,-34.669777,auto-pasajeros,auto,pasajeros
2,2016-0003,1,2016-01-03,7.0,avenida,1.0,-58.390403,-34.631894,moto-auto,moto,auto
3,2016-0004,1,2016-01-10,0.0,avenida,8.0,-58.465039,-34.680930,moto-sd,moto,nan
4,2016-0005,1,2016-01-21,5.0,avenida,1.0,-58.387183,-34.622466,moto-pasajeros,moto,pasajeros


In [324]:
# Convertimos los nan a valores nulos
homicidios_hechos.replace('nan', np.nan, inplace=True)

Creamos la columna 'gravedad' y asignamos el valor 'fatal'

In [325]:
homicidios_hechos['gravedad']='fatal'

In [326]:
homicidios_hechos.head()

,id_hecho,n_victimas,fecha,hh,tipo_calle,comuna,longitud,latitud,participantes,victima,acusado,gravedad
0,2016-0001,1,2016-01-01,4.0,avenida,8.0,-58.475340,-34.687570,moto-auto,moto,auto,fatal
1,2016-0002,1,2016-01-02,1.0,gral paz,9.0,-58.508775,-34.669777,auto-pasajeros,auto,pasajeros,fatal
2,2016-0003,1,2016-01-03,7.0,avenida,1.0,-58.390403,-34.631894,moto-auto,moto,auto,fatal
3,2016-0004,1,2016-01-10,0.0,avenida,8.0,-58.465039,-34.680930,moto-sd,moto,NaN,fatal
4,2016-0005,1,2016-01-21,5.0,avenida,1.0,-58.387183,-34.622466,moto-pasajeros,moto,pasajeros,fatal


Cambiar el contenido de la columna **victima**, según el diccionario dado.

In [327]:
mapeo_correcciones_victima = {'peaton': 'peatón','movil': 'móvil'}
homicidios_hechos['victima'] = homicidios_hechos['victima'].replace(mapeo_correcciones_victima)

In [328]:
homicidios_hechos.head()

,id_hecho,n_victimas,fecha,hh,tipo_calle,comuna,longitud,latitud,participantes,victima,acusado,gravedad
0,2016-0001,1,2016-01-01,4.0,avenida,8.0,-58.475340,-34.687570,moto-auto,moto,auto,fatal
1,2016-0002,1,2016-01-02,1.0,gral paz,9.0,-58.508775,-34.669777,auto-pasajeros,auto,pasajeros,fatal
2,2016-0003,1,2016-01-03,7.0,avenida,1.0,-58.390403,-34.631894,moto-auto,moto,auto,fatal
3,2016-0004,1,2016-01-10,0.0,avenida,8.0,-58.465039,-34.680930,moto-sd,moto,NaN,fatal
4,2016-0005,1,2016-01-21,5.0,avenida,1.0,-58.387183,-34.622466,moto-pasajeros,moto,pasajeros,fatal


In [329]:
resumen_tipos_de_datos(homicidios_hechos)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,id_hecho,[str],0,0.00
1,n_victimas,[int],0,0.00
2,fecha,[Timestamp],0,0.00
3,hh,"[float, nan]",1,0.14
4,tipo_calle,[str],0,0.00
5,comuna,"[float, nan]",2,0.29
6,longitud,[float],12,1.72
7,latitud,[float],12,1.72
8,participantes,[str],0,0.00
9,victima,"[str, float]",9,1.29


# ETL dataframe ``HOMICIDIOS-VICTIMAS``

In [330]:
df_homicidios_victimas= pd.read_excel('../datasets/raw/homicidios.xlsx',sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])

In [331]:
df_homicidios_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00


### 1. Eliminación de columnas
La columna `FECHA` será conservada, las columnas **AAAA**, **DD**, y **MM**, serán eliminadas por ser consideradas información redundate.

In [332]:
df_homicidios_victimas.drop(columns=['AAAA','MM','DD','FECHA_FALLECIMIENTO'],inplace=True)

### 2. Valores nulos

In [333]:
resumen_tipos_de_datos(df_homicidios_victimas)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,ID_hecho,[str],0,0.00
1,FECHA,[Timestamp],0,0.00
2,ROL,"[str, float]",11,1.53
3,VICTIMA,"[str, float]",9,1.26
4,SEXO,"[str, float]",6,0.84
5,EDAD,[float],53,7.39


### 3. Cambio de tipo de datos
Conversión de columnas ``ROL``, ``VICTIMA`` y ``SEXO`` a tipo categórico

In [334]:
df_homicidios_victimas['ROL'] = df_homicidios_victimas['ROL'].astype('category')
df_homicidios_victimas['VICTIMA'] = df_homicidios_victimas['VICTIMA'].astype('category')
df_homicidios_victimas['SEXO'] = df_homicidios_victimas['SEXO'].astype('category')

### 4. Renombrar columnas
Todos los nombres de las columnas serán convertidos a minúsculas. 

In [335]:
df_homicidios_victimas.columns = df_homicidios_victimas.columns.str.lower()

Cambiamos el nombre de las columnas

In [336]:
df_homicidios_victimas.rename(columns={'edad': 'edad_victima','victima':'vehiculo_victima','sexo':'sexo_victima'}, inplace=True)

A las columnas 'rol', 'victima', 'sexo' y 'acusado' se les cambió el contenido de sus filas a minúsculas.

In [337]:
columnas_a_minusc = ['rol', 'vehiculo_victima', 'sexo_victima']
df_homicidios_victimas[columnas_a_minusc] = df_homicidios_victimas[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

In [338]:
df_homicidios_victimas

,id_hecho,fecha,rol,vehiculo_victima,sexo_victima,edad_victima
0,2016-0001,2016-01-01,conductor,moto,masculino,19.0
1,2016-0002,2016-01-02,conductor,auto,masculino,70.0
2,2016-0003,2016-01-03,conductor,moto,masculino,30.0
3,2016-0004,2016-01-10,conductor,moto,masculino,18.0
4,2016-0005,2016-01-21,conductor,moto,masculino,29.0
...,...,...,...,...,...,...
712,2021-0092,2021-12-12,peaton,peaton,femenino,50.0
713,2021-0093,2021-12-13,pasajero_acompañante,moto,femenino,18.0
714,2021-0094,2021-12-20,pasajero_acompañante,moto,femenino,43.0
715,2021-0095,2021-12-30,conductor,moto,masculino,27.0


In [339]:
df_homicidios_victimas['rol'].unique()

array(['conductor', 'pasajero_acompañante', 'peaton', 'nan', 'ciclista'],
      dtype=object)

In [340]:
df_homicidios_victimas['vehiculo_victima'].unique()

array(['moto', 'auto', 'peaton', 'nan', 'cargas', 'bicicleta',
       'pasajeros', 'movil'], dtype=object)

Cambiamos los datos de las columnas ``rol`` y ``vehiculo_victima``, siguiendo las indicaciones del diccionario

In [341]:
# Diccionario de correcciones
diccionario_correcciones_rol = {'pasajero_acompañante': 'pasajero', 'peaton': 'peatón'}
diccionario_correcciones_vehi_vic = {'peaton': 'peatón', 'movil': 'móvil'}
df_homicidios_victimas['rol'] = df_homicidios_victimas['rol'].replace(diccionario_correcciones_rol)
df_homicidios_victimas['vehiculo_victima'] = df_homicidios_victimas['vehiculo_victima'].replace(diccionario_correcciones_vehi_vic)

In [342]:
df_homicidios_victimas['gravedad'] = 'fatal'

In [343]:
df_homicidios_victimas.head()

,id_hecho,fecha,rol,vehiculo_victima,sexo_victima,edad_victima,gravedad
0,2016-0001,2016-01-01,conductor,moto,masculino,19.0,fatal
1,2016-0002,2016-01-02,conductor,auto,masculino,70.0,fatal
2,2016-0003,2016-01-03,conductor,moto,masculino,30.0,fatal
3,2016-0004,2016-01-10,conductor,moto,masculino,18.0,fatal
4,2016-0005,2016-01-21,conductor,moto,masculino,29.0,fatal


In [344]:
df_homicidios_victimas['RANGO_ETARIO'] = df_homicidios_victimas['edad_victima'].apply(lambda x: asignar_rango_etario(x))
df_homicidios_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_hecho          717 non-null    object        
 1   fecha             717 non-null    datetime64[ns]
 2   rol               717 non-null    object        
 3   vehiculo_victima  717 non-null    object        
 4   sexo_victima      717 non-null    object        
 5   edad_victima      664 non-null    float64       
 6   gravedad          717 non-null    object        
 7   RANGO_ETARIO      664 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 44.9+ KB


# MERGE DE LOS DATAFRAMES

### DataFrame ``homicidios_hechos`` y ``homicidios_víctimas``

Integración de Datos: Fusión de Información sobre Homicidios Hechos y Víctimas

In [345]:
df_homicidios_combinado = pd.merge(homicidios_hechos, df_homicidios_victimas, how='inner', on='id_hecho')

In [346]:
df_homicidios_combinado

,id_hecho,n_victimas,fecha_x,hh,tipo_calle,comuna,longitud,latitud,participantes,victima,acusado,gravedad_x,fecha_y,rol,vehiculo_victima,sexo_victima,edad_victima,gravedad_y,RANGO_ETARIO
0,2016-0001,1,2016-01-01,4.0,avenida,8.0,-58.475340,-34.687570,moto-auto,moto,auto,fatal,2016-01-01,conductor,moto,masculino,19.0,fatal,2_ 18 a 29 años
1,2016-0002,1,2016-01-02,1.0,gral paz,9.0,-58.508775,-34.669777,auto-pasajeros,auto,pasajeros,fatal,2016-01-02,conductor,auto,masculino,70.0,fatal,6_ 60 años o más
2,2016-0003,1,2016-01-03,7.0,avenida,1.0,-58.390403,-34.631894,moto-auto,moto,auto,fatal,2016-01-03,conductor,moto,masculino,30.0,fatal,3_ 30 a 39 años
3,2016-0004,1,2016-01-10,0.0,avenida,8.0,-58.465039,-34.680930,moto-sd,moto,NaN,fatal,2016-01-10,conductor,moto,masculino,18.0,fatal,2_ 18 a 29 años
4,2016-0005,1,2016-01-21,5.0,avenida,1.0,-58.387183,-34.622466,moto-pasajeros,moto,pasajeros,fatal,2016-01-21,conductor,moto,masculino,29.0,fatal,2_ 18 a 29 años
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,17.0,avenida,7.0,-58.433538,-34.645616,moto-auto,moto,auto,fatal,2021-12-13,pasajero,moto,femenino,18.0,fatal,2_ 18 a 29 años
713,2021-0094,1,2021-12-20,1.0,autopista,9.0,-58.467398,-34.651178,moto-auto,moto,auto,fatal,2021-12-20,pasajero,moto,femenino,43.0,fatal,4_ 40 a 49 años
714,2021-0095,1,2021-12-30,0.0,avenida,11.0,-58.472934,-34.619847,moto-cargas,moto,cargas,fatal,2021-12-30,conductor,moto,masculino,27.0,fatal,2_ 18 a 29 años
715,2021-0096,1,2021-12-15,10.0,avenida,9.0,-58.470668,-34.650217,auto-cargas,auto,cargas,fatal,2021-12-15,conductor,auto,masculino,60.0,fatal,6_ 60 años o más


Refinamiento de Datos: Ajuste de columnas en el conjunto de datos de Homicidios Combinado, según el diccionario dado.

In [347]:
df_homicidios_combinado.drop(columns=['fecha_y', 'gravedad_y'], inplace=True)
df_homicidios_combinado.rename(columns={
    'fecha_x': 'fecha',
    'gravedad_x': 'gravedad',
    'hh': 'franja_hora',
    'longitud': 'pos_x',
    'latitud': 'pos_y',
    'vehiculo_victima': 'vehiculo_victima',  # Asumiendo que deseas conservar este nombre
    'sexo_victima': 'sexo_victima',
    'edad_victima': 'edad_victima'
    }, inplace=True)

In [348]:
df_homicidios_combinado

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,victima,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima,RANGO_ETARIO
0,2016-0001,1,2016-01-01,4.0,avenida,8.0,-58.475340,-34.687570,moto-auto,moto,auto,fatal,conductor,moto,masculino,19.0,2_ 18 a 29 años
1,2016-0002,1,2016-01-02,1.0,gral paz,9.0,-58.508775,-34.669777,auto-pasajeros,auto,pasajeros,fatal,conductor,auto,masculino,70.0,6_ 60 años o más
2,2016-0003,1,2016-01-03,7.0,avenida,1.0,-58.390403,-34.631894,moto-auto,moto,auto,fatal,conductor,moto,masculino,30.0,3_ 30 a 39 años
3,2016-0004,1,2016-01-10,0.0,avenida,8.0,-58.465039,-34.680930,moto-sd,moto,NaN,fatal,conductor,moto,masculino,18.0,2_ 18 a 29 años
4,2016-0005,1,2016-01-21,5.0,avenida,1.0,-58.387183,-34.622466,moto-pasajeros,moto,pasajeros,fatal,conductor,moto,masculino,29.0,2_ 18 a 29 años
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,17.0,avenida,7.0,-58.433538,-34.645616,moto-auto,moto,auto,fatal,pasajero,moto,femenino,18.0,2_ 18 a 29 años
713,2021-0094,1,2021-12-20,1.0,autopista,9.0,-58.467398,-34.651178,moto-auto,moto,auto,fatal,pasajero,moto,femenino,43.0,4_ 40 a 49 años
714,2021-0095,1,2021-12-30,0.0,avenida,11.0,-58.472934,-34.619847,moto-cargas,moto,cargas,fatal,conductor,moto,masculino,27.0,2_ 18 a 29 años
715,2021-0096,1,2021-12-15,10.0,avenida,9.0,-58.470668,-34.650217,auto-cargas,auto,cargas,fatal,conductor,auto,masculino,60.0,6_ 60 años o más


In [349]:
# Eliminar la columna 'vehiculo_victima' de forma permanente
df_homicidios_combinado.drop('victima', axis=1, inplace=True)

Convertir las columnas a ``enteros`` y manejar los valores NaN

In [350]:
df_homicidios_combinado['franja_hora'] = df_homicidios_combinado['franja_hora'].astype('Int64')
df_homicidios_combinado['edad_victima'] = df_homicidios_combinado['edad_victima'].astype('Int64')

Convertir a tipo de datos ``categóricos``

In [351]:
df_homicidios_combinado['tipo_calle'] = df_homicidios_combinado['tipo_calle'].astype('category')
df_homicidios_combinado['comuna'] = df_homicidios_combinado['comuna'].astype('category')
df_homicidios_combinado['participantes'] = df_homicidios_combinado['participantes'].astype('category')
df_homicidios_combinado['acusado'] = df_homicidios_combinado['acusado'].astype('category')
df_homicidios_combinado['gravedad'] = df_homicidios_combinado['gravedad'].astype('category')
df_homicidios_combinado['rol'] = df_homicidios_combinado['rol'].astype('category')
df_homicidios_combinado['vehiculo_victima'] = df_homicidios_combinado['vehiculo_victima'].astype('category')
df_homicidios_combinado['sexo_victima'] = df_homicidios_combinado['sexo_victima'].astype('category')

Convertimos a tipo de dato ``datetime``

In [352]:
df_homicidios_combinado['fecha'] = pd.to_datetime(df_homicidios_combinado['fecha'], errors='coerce')

In [353]:
df_homicidios_combinado.to_csv('../datasets/homicidios_hechos_victimas.csv', index=False)

#### Tratando columnas ``franja_hora`` y `comuna`

In [354]:
nombre_columna = 'franja_hora'
filas_con_nulos = df_homicidios_combinado[df_homicidios_combinado[nombre_columna].isnull()]
filas_con_nulos

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima,RANGO_ETARIO
536,2019-0103,1,2019-12-18,<NA>,gral paz,11.0,-58.521694,-34.594716,moto-moto,moto,fatal,conductor,moto,masculino,24,2_ 18 a 29 años


Procedemos a imputar con la ``mediana`` el dato faltante en la columna ``franja_hora``, porque preferimos preservar la fila completa, ya que ese único dato faltante no afectará en gran manera

In [355]:
mediana_edad = df_homicidios_combinado['franja_hora'].median()
df_homicidios_combinado['franja_hora'] = df_homicidios_combinado['franja_hora'].fillna(mediana_edad)

In [356]:
nombre_columna = 'comuna'
filas_con_nulos = df_homicidios_combinado[df_homicidios_combinado[nombre_columna].isnull()]
filas_con_nulos

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima,RANGO_ETARIO
121,2016-0151,1,2016-11-18,20,calle,NaN,NaN,NaN,peaton-sd,NaN,fatal,peatón,peatón,nan,<NA>,NaN
141,2016-0174,1,2016-12-27,0,autopista,NaN,NaN,NaN,sd-sd,NaN,fatal,nan,nan,nan,<NA>,NaN


Procedemos a eliminar estas 2 filas, ya que la información en su mayoría de columnas es nula, no aportará mucho preservar estas columnas

In [357]:
filas_a_eliminar = [121, 141]
df_homicidios_combinado =df_homicidios_combinado.drop(filas_a_eliminar, axis=0)

In [358]:
df_homicidios_combinado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_hecho          715 non-null    object        
 1   n_victimas        715 non-null    int64         
 2   fecha             715 non-null    datetime64[ns]
 3   franja_hora       715 non-null    Int64         
 4   tipo_calle        715 non-null    category      
 5   comuna            715 non-null    category      
 6   pos_x             704 non-null    float64       
 7   pos_y             704 non-null    float64       
 8   participantes     715 non-null    category      
 9   acusado           694 non-null    category      
 10  gravedad          715 non-null    category      
 11  rol               715 non-null    category      
 12  vehiculo_victima  715 non-null    category      
 13  sexo_victima      715 non-null    category      
 14  edad_victima      664 non-null 

#### Tratando columna `pos_x` y `pos_y`
Verificamos de manera general la información que tiene todas las columnas, cuya columna `pos_x` es nulo

In [359]:
nombre_columna = 'pos_x'
filas_con_nulos = df_homicidios_combinado[df_homicidios_combinado[nombre_columna].isnull()]
filas_con_nulos.head()

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima,RANGO_ETARIO
39,2016-0052,1,2016-04-20,20,autopista,13.0,NaN,NaN,moto-sd,NaN,fatal,nan,moto,nan,<NA>,NaN
108,2016-0136,1,2016-10-25,0,autopista,4.0,NaN,NaN,moto-cargas,cargas,fatal,conductor,moto,nan,<NA>,NaN
182,2017-0042,1,2017-04-10,9,gral paz,14.0,NaN,NaN,moto-cargas,cargas,fatal,conductor,moto,masculino,<NA>,NaN
186,2017-0050,2,2017-04-28,11,autopista,9.0,NaN,NaN,moto-cargas,cargas,fatal,conductor,moto,masculino,46,4_ 40 a 49 años
187,2017-0050,2,2017-04-28,11,autopista,9.0,NaN,NaN,moto-cargas,cargas,fatal,pasajero,moto,masculino,16,1_ 0 a 17 años


Obteniendo la lista de códigos ``id_hecho`` donde las filas son nulas en **pos_x** y **pos_y**

In [360]:
nombre_columna = 'pos_x'
filas_con_nulos = df_homicidios_combinado[df_homicidios_combinado[nombre_columna].isnull()]
codigos_con_nulos = filas_con_nulos['id_hecho'].tolist()
print(codigos_con_nulos)

['2016-0052', '2016-0136', '2017-0042', '2017-0050', '2017-0050', '2017-0051', '2017-0140', '2018-0039', '2020-0026', '2020-0039', '2021-0023']


Traemos la base original, para poder obtener coordenas, usando la columna ``LUGAR_DEL_HECHO``, con la herramienta API de google

In [361]:
df_original = pd.read_excel('../datasets/raw/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])

In [362]:
# Normalizamos los id_hecho, para ambos DF
df_original.rename(columns={'ID': 'id_hecho'}, inplace=True)

In [363]:
# Filtrar la base de datos original usando los códigos con nulos
df_original_filtrado = df_original[df_original['id_hecho'].isin(codigos_con_nulos)]
df_original_filtrado.head()

,id_hecho,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20.0,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN,13,Point (. .),.,.,MOTO-SD,MOTO,NaN
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0.0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9.0,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",NaN,NaN,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11.0,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,NaN,NaN,NaN,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3.0,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN,NaN,NaN,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO


Geocodificación automatizada de coordenadas a partir de direcciones

In [364]:
gmaps = googlemaps.Client(key='Holaaa :)')

def actualizar_coordenadas_con_api(df_hhv, df_original):
    for index, row in df_hhv.iterrows():
        if pd.isna(row['pos_x']) or pd.isna(row['pos_y']):
            direccion_original = df_original.loc[df_original['id_hecho'] == row['id_hecho'], 'LUGAR_DEL_HECHO'].values[0]
            geocode_result = gmaps.geocode(direccion_original)
            if geocode_result:
                lat = geocode_result[0]['geometry']['location']['lng']
                lng = geocode_result[0]['geometry']['location']['lat']
                df_hhv.at[index, 'pos_x'] = lng
                df_hhv.at[index, 'pos_y'] = lat
            else:
                print(f"No se pudo obtener la ubicación para el id_hecho {row['id_hecho']}")

Corroboramos cuantos datos se pudieron obtener sus coordenas y cuales faltaron

In [365]:
actualizar_coordenadas_con_api(df_homicidios_combinado, df_original)

No se pudo obtener la ubicación para el id_hecho 2016-0052
No se pudo obtener la ubicación para el id_hecho 2017-0042


No se pudo obtener la ubicación para el id_hecho 2018-0039
No se pudo obtener la ubicación para el id_hecho 2020-0026


### Tratamos la columna `acusado`

In [366]:
if 'Desconocido' not in df_homicidios_combinado['acusado'].cat.categories:
    df_homicidios_combinado['acusado'] = df_homicidios_combinado['acusado'].cat.add_categories('Desconocido')

df_homicidios_combinado['acusado'] = df_homicidios_combinado['acusado'].fillna('Desconocido')
df_homicidios_combinado['acusado'].value_counts()

acusado
auto           210
pasajeros      178
cargas         150
objeto fijo     67
moto            58
Desconocido     21
multiple        17
bicicleta        7
otro             6
tren             1
Name: count, dtype: int64

### Tratamos las columna `rol`, `vehiculo_victima`, `sexo_victima`

In [367]:
columnas_categoricas = ['rol', 'vehiculo_victima', 'sexo_victima']

for columna in columnas_categoricas:
    if 'Desconocido' not in df_homicidios_combinado[columna].cat.categories:
        df_homicidios_combinado[columna] = df_homicidios_combinado[columna].cat.add_categories('Desconocido')
    
    df_homicidios_combinado[columna] = df_homicidios_combinado[columna].fillna('Desconocido')

In [368]:
df_homicidios_combinado.head()

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima,RANGO_ETARIO
0,2016-0001,1,2016-01-01,4,avenida,8.0,-58.475340,-34.687570,moto-auto,auto,fatal,conductor,moto,masculino,19,2_ 18 a 29 años
1,2016-0002,1,2016-01-02,1,gral paz,9.0,-58.508775,-34.669777,auto-pasajeros,pasajeros,fatal,conductor,auto,masculino,70,6_ 60 años o más
2,2016-0003,1,2016-01-03,7,avenida,1.0,-58.390403,-34.631894,moto-auto,auto,fatal,conductor,moto,masculino,30,3_ 30 a 39 años
3,2016-0004,1,2016-01-10,0,avenida,8.0,-58.465039,-34.680930,moto-sd,Desconocido,fatal,conductor,moto,masculino,18,2_ 18 a 29 años
4,2016-0005,1,2016-01-21,5,avenida,1.0,-58.387183,-34.622466,moto-pasajeros,pasajeros,fatal,conductor,moto,masculino,29,2_ 18 a 29 años


In [369]:
df_homicidios_combinado.to_csv('../datasets/lesiones_hechos_victimas.csv', index=False)

## DataFrame ``lesiones_hechos`` y ``lesiones_víctimas``

Integración de Datos: Fusión de Información sobre Homicidios Hechos y Víctimas

In [370]:
df_final_lesiones = pd.merge(df_lesiones_hechos, df_lesiones_victimas, on='id_hecho', how='inner')

In [371]:
df_final_lesiones

,id_hecho,n_victimas,fecha_x,hh,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,longitud,latitud,victima,acusado,participantes,gravedad_x,fecha_y,vehiculo_victima,sexo_victima,edad_victima,gravedad_y,RANGO_ETARIO
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-5.840891e+01,-3.455966e+01,ciclista,NaN,ciclista-sd,leve,2019-01-01,NaN,NaN,<NA>,leve,NaN
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-5.844351e+01,-3.466913e+01,auto,NaN,auto-sd,leve,2019-01-01,NaN,Masculino,57,leve,5_ 50 a 59 años
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-5.846834e+01,-3.467756e+01,NaN,NaN,sd-sd,leve,2019-01-01,NaN,NaN,<NA>,leve,NaN
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-5.843742e+01,-3.464735e+01,peatón,NaN,peaton-sd,leve,2019-01-01,NaN,Masculino,<NA>,leve,NaN
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-5.839822e+01,-3.460458e+01,auto,NaN,auto-sd,leve,2019-01-01,NaN,Masculino,45,leve,4_ 40 a 49 años
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27593,LC-2021-0652865,2,2021-12-31,19,"ALBERDI, JUAN BAUTISTA AV. 4436",9,avenida,NaN,"ALBERDI, JUAN BAUTISTA AV.",4436.0,NaN,Point (97705.4491469278 98614.8825470092),-5.848833e+01,-3.464175e+01,NaN,NaN,sd-sd,leve,2021-12-31,NaN,Masculino,28,leve,2_ 18 a 29 años
27594,LC-2021-0652865,2,2021-12-31,19,"ALBERDI, JUAN BAUTISTA AV. 4436",9,avenida,NaN,"ALBERDI, JUAN BAUTISTA AV.",4436.0,NaN,Point (97705.4491469278 98614.8825470092),-5.848833e+01,-3.464175e+01,NaN,NaN,sd-sd,leve,2021-12-31,NaN,Masculino,52,leve,5_ 50 a 59 años
27595,LC-2021-0652907,1,2021-12-31,20,NaN,1,NaN,Barrio Padre Carlos Mugica (Villa 31 bis),NaN,NaN,NaN,Point (107377.10694920316745993 105120.7873965...,-5.838289e+01,-3.458308e+01,NaN,NaN,sd-sd,leve,2021-12-31,NaN,Masculino,23,leve,2_ 18 a 29 años
27596,LC-2021-0652921,1,2021-12-31,22,LINIERS VIRREY y MORENO,5,calle,NaN,LINIERS VIRREY,0.0,MORENO,Point (104472.702045632 101660.930364258),-5.841453e+01,-3.461429e+01,moto,transporte publico,moto-transporte publico,grave,2021-12-31,MOTO,Masculino,31,GRAVE,3_ 30 a 39 años


In [372]:
# Eliminar columnas redundantes
df_final_lesiones = df_final_lesiones.drop(columns=['fecha_y', 'gravedad_y'])

# Renombrar columnas
df_final_lesiones.rename(columns={
    'fecha_x': 'fecha',
    'hh': 'franja_hora',
    'gravedad_x': 'gravedad',
    'longitud': 'pos_x',
    'latitud': 'pos_y',
}, inplace=True)

In [373]:
df_final_lesiones.head()

,id_hecho,n_victimas,fecha,franja_hora,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,pos_x,pos_y,victima,acusado,participantes,gravedad,vehiculo_victima,sexo_victima,edad_victima,RANGO_ETARIO
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-58.408911,-34.559658,ciclista,NaN,ciclista-sd,leve,NaN,NaN,<NA>,NaN
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-58.443510,-34.669125,auto,NaN,auto-sd,leve,NaN,Masculino,57,5_ 50 a 59 años
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-58.468335,-34.677556,NaN,NaN,sd-sd,leve,NaN,NaN,<NA>,NaN
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-58.437425,-34.647349,peatón,NaN,peaton-sd,leve,NaN,Masculino,<NA>,NaN
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-58.398225,-34.604579,auto,NaN,auto-sd,leve,NaN,Masculino,45,4_ 40 a 49 años


In [374]:
resumen_tipos_de_datos(df_final_lesiones)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,id_hecho,[str],0,0.00
1,n_victimas,[int],0,0.00
2,fecha,[Timestamp],0,0.00
3,franja_hora,[float],8,0.03
4,direccion_normalizada,"[float, str]",12696,46.00
5,comuna,[float],1624,5.88
6,tipo_calle,"[float, str]",12879,46.67
7,otra_direccion,"[str, float]",21138,76.59
8,calle,"[float, str]",12744,46.18
9,altura,[float],12847,46.55


In [375]:
nombre_columna = 'pos_x'
filas_con_nulos = df_final_lesiones[df_final_lesiones[nombre_columna].isnull()]
codigos_con_nulos = filas_con_nulos['id_hecho'].tolist()
print(codigos_con_nulos)

['LC-2019-0000928', 'LC-2019-0022496', 'LC-2019-0007634', 'LC-2019-0008283', 'LC-2019-0008283', 'LC-2019-0008974', 'LC-2019-0010983', 'LC-2019-0010983', 'LC-2019-0011092', 'LC-2019-0011335', 'LC-2019-0017186', 'LC-2019-0022256', 'LC-2019-0687058', 'LC-2019-0688515', 'LC-2019-0757409', 'LC-2019-0025134', 'LC-2019-0025021', 'LC-2019-0029566', 'LC-2019-0033845', 'LC-2019-0034229', 'LC-2019-0034229', 'LC-2019-0034229', 'LC-2019-0034229', 'LC-2019-0033624', 'LC-2019-0036508', 'LC-2019-0037956', 'LC-2019-0037956', 'LC-2019-0039050', 'LC-2019-0039585', 'LC-2019-0044699', 'LC-2019-0047479', 'LC-2019-0047733', 'LC-2019-0049828', 'LC-2019-0073279', 'LC-2019-0049592', 'LC-2019-0050069', 'LC-2019-0050175', 'LC-2019-0050175', 'LC-2019-0050670', 'LC-2019-0051216', 'LC-2019-0051464', 'LC-2019-0051464', 'LC-2019-0051508', 'LC-2019-0053244', 'LC-2019-0056387', 'LC-2019-0056420', 'LC-2019-0056420', 'LC-2019-0057074', 'LC-2019-0057299', 'LC-2019-0059155', 'LC-2019-0060398', 'LC-2019-0060398', 'LC-2019-00

In [376]:
df_original = pd.read_excel('../datasets/raw/lesiones.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])
# Normalizamos los id_hecho, para ambos DF
df_original.rename(columns={'id': 'id_hecho'}, inplace=True)

In [377]:
gmaps = googlemaps.Client(key='Holaaaa :)')

def actualizar_coordenadas_con_api(df_final_lesiones, df_original):
    for index, row in df_final_lesiones.iterrows():
        if pd.isna(row['pos_x']) or pd.isna(row['pos_y']):
            direccion_original = df_original.loc[df_original['id_hecho'] == row['id_hecho'], 'direccion_normalizada'].values[0]
            geocode_result = gmaps.geocode(direccion_original)
            if geocode_result:
                lat = geocode_result[0]['geometry']['location']['lng']
                lng = geocode_result[0]['geometry']['location']['lat']
                df_final_lesiones.at[index, 'pos_x'] = lng
                df_final_lesiones.at[index, 'pos_y'] = lat
            else:
                print(f"No se pudo obtener la ubicación para el id_hecho {row['id_hecho']}")

In [379]:
actualizar_coordenadas_con_api(df_final_lesiones, df_original)

No se pudo obtener la ubicación para el id_hecho LC-2019-0000928
No se pudo obtener la ubicación para el id_hecho LC-2019-0022496
No se pudo obtener la ubicación para el id_hecho LC-2019-0007634
No se pudo obtener la ubicación para el id_hecho LC-2019-0008283
No se pudo obtener la ubicación para el id_hecho LC-2019-0008283
No se pudo obtener la ubicación para el id_hecho LC-2019-0008974
No se pudo obtener la ubicación para el id_hecho LC-2019-0010983
No se pudo obtener la ubicación para el id_hecho LC-2019-0010983
No se pudo obtener la ubicación para el id_hecho LC-2019-0011092
No se pudo obtener la ubicación para el id_hecho LC-2019-0011335
No se pudo obtener la ubicación para el id_hecho LC-2019-0017186
No se pudo obtener la ubicación para el id_hecho LC-2019-0022256
No se pudo obtener la ubicación para el id_hecho LC-2019-0687058
No se pudo obtener la ubicación para el id_hecho LC-2019-0025134
No se pudo obtener la ubicación para el id_hecho LC-2019-0029566
No se pudo obtener la ubi

In [380]:
resumen_tipos_de_datos(df_final_lesiones)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,id_hecho,[str],0,0.00
1,n_victimas,[int],0,0.00
2,fecha,[Timestamp],0,0.00
3,franja_hora,[float],8,0.03
4,direccion_normalizada,"[float, str]",12696,46.00
5,comuna,[float],1624,5.88
6,tipo_calle,"[float, str]",12879,46.67
7,otra_direccion,"[str, float]",21138,76.59
8,calle,"[float, str]",12744,46.18
9,altura,[float],12847,46.55


In [381]:
nombre_columna = 'pos_x'
filas_con_nulos2 = df_final_lesiones[df_final_lesiones[nombre_columna].isnull()]
codigos_con_nulos2 = filas_con_nulos2['id_hecho'].tolist()
print(codigos_con_nulos2)

['LC-2019-0000928', 'LC-2019-0022496', 'LC-2019-0007634', 'LC-2019-0008283', 'LC-2019-0008283', 'LC-2019-0008974', 'LC-2019-0010983', 'LC-2019-0010983', 'LC-2019-0011092', 'LC-2019-0011335', 'LC-2019-0017186', 'LC-2019-0022256', 'LC-2019-0687058', 'LC-2019-0025134', 'LC-2019-0029566', 'LC-2019-0033845', 'LC-2019-0034229', 'LC-2019-0034229', 'LC-2019-0034229', 'LC-2019-0034229', 'LC-2019-0036508', 'LC-2019-0037956', 'LC-2019-0037956', 'LC-2019-0039050', 'LC-2019-0039585', 'LC-2019-0044699', 'LC-2019-0047479', 'LC-2019-0047733', 'LC-2019-0049828', 'LC-2019-0073279', 'LC-2019-0049592', 'LC-2019-0050069', 'LC-2019-0050175', 'LC-2019-0050175', 'LC-2019-0050670', 'LC-2019-0051216', 'LC-2019-0051464', 'LC-2019-0051464', 'LC-2019-0051508', 'LC-2019-0053244', 'LC-2019-0056387', 'LC-2019-0056420', 'LC-2019-0056420', 'LC-2019-0057074', 'LC-2019-0057299', 'LC-2019-0059155', 'LC-2019-0060398', 'LC-2019-0060398', 'LC-2019-0064934', 'LC-2019-0061593', 'LC-2019-0068407', 'LC-2019-0064712', 'LC-2019-00

In [382]:
gmaps = googlemaps.Client(key='Holaaa :3')

def actualizar_coordenadas_con_api(df_final_lesiones, df_original):
    for index, row in df_final_lesiones.iterrows():
        if pd.isna(row['pos_x']) or pd.isna(row['pos_y']):
            direccion_original = df_original.loc[df_original['id_hecho'] == row['id_hecho'], 'otra_direccion'].values[0]
            geocode_result = gmaps.geocode(direccion_original)
            if geocode_result:
                lat = geocode_result[0]['geometry']['location']['lng']
                lng = geocode_result[0]['geometry']['location']['lat']
                df_final_lesiones.at[index, 'pos_x'] = lng
                df_final_lesiones.at[index, 'pos_y'] = lat
            else:
                print(f"No se pudo obtener la ubicación para el id_hecho {row['id_hecho']}")

In [383]:
actualizar_coordenadas_con_api(df_final_lesiones, df_original)

No se pudo obtener la ubicación para el id_hecho LC-2019-0007634
No se pudo obtener la ubicación para el id_hecho LC-2019-0008974
No se pudo obtener la ubicación para el id_hecho LC-2019-0010983
No se pudo obtener la ubicación para el id_hecho LC-2019-0010983
No se pudo obtener la ubicación para el id_hecho LC-2019-0011092
No se pudo obtener la ubicación para el id_hecho LC-2019-0017186
No se pudo obtener la ubicación para el id_hecho LC-2019-0022256
No se pudo obtener la ubicación para el id_hecho LC-2019-0687058
No se pudo obtener la ubicación para el id_hecho LC-2019-0033845
No se pudo obtener la ubicación para el id_hecho LC-2019-0034229
No se pudo obtener la ubicación para el id_hecho LC-2019-0034229
No se pudo obtener la ubicación para el id_hecho LC-2019-0034229
No se pudo obtener la ubicación para el id_hecho LC-2019-0034229
No se pudo obtener la ubicación para el id_hecho LC-2019-0036508
No se pudo obtener la ubicación para el id_hecho LC-2019-0037956
No se pudo obtener la ubi

In [ ]:
resumen_tipos_de_datos(df_final_lesiones)

,Columna,Tipos de Dato,Cantidad de Nulos,% de Nulos
0,id_hecho,[str],0,0.00
1,n_victimas,[int],0,0.00
2,fecha,[Timestamp],0,0.00
3,franja_hora,[float],8,0.03
4,direccion_normalizada,"[float, str]",12696,46.00
5,comuna,[float],1624,5.88
6,tipo_calle,"[float, str]",12879,46.67
7,otra_direccion,"[str, float]",21138,76.59
8,calle,"[float, str]",12744,46.18
9,altura,[float],12847,46.55


In [387]:
df_final_lesiones.head()

,id_hecho,n_victimas,fecha,franja_hora,direccion_normalizada,comuna,tipo_calle,otra_direccion,calle,altura,cruce,geocodificacion_CABA,pos_x,pos_y,victima,acusado,participantes,gravedad,vehiculo_victima,sexo_victima,edad_victima,RANGO_ETARIO
0,LC-2019-0000179,1,2019-01-01,9,NaN,14,NaN,"Obligado Rafael, Av.Costanera 3300, Barrio: Pa...",NaN,NaN,NaN,Point (104991.46333918849995825 107720.9664748...,-58.408911,-34.559658,ciclista,NaN,ciclista-sd,leve,NaN,NaN,<NA>,NaN
1,LC-2019-0000053,1,2019-01-01,1,NaN,8,NaN,"Autopista Presidente Héctor J. Cámpora y Roca,...",NaN,NaN,NaN,Point (101813.84712503915943671 95578.55507230...,-58.443510,-34.669125,auto,NaN,auto-sd,leve,NaN,Masculino,57,5_ 50 a 59 años
2,LC-2019-0000063,1,2019-01-01,2,NaN,8,NaN,"Barros Pazos, Jose y Larraya 0, Barrio: Villa ...",NaN,NaN,NaN,Point (99538.58660300650808495 94643.418717990...,-58.468335,-34.677556,NaN,NaN,sd-sd,leve,NaN,NaN,<NA>,NaN
3,LC-2019-0000079,1,2019-01-01,2,NaN,7,NaN,"Torres Y Tenorio, Pres. y Riestra Av. 0, Barri...",NaN,NaN,NaN,Point (102372.16830963784013875 97994.13750300...,-58.437425,-34.647349,peatón,NaN,peaton-sd,leve,NaN,Masculino,<NA>,NaN
4,LC-2019-0000082,4,2019-01-01,4,NaN,3,NaN,"Corrientes Av. y Uriburu Jose E., Pres. 0, Bar...",NaN,NaN,NaN,Point (105968.98286849579017144 102737.1734686...,-58.398225,-34.604579,auto,NaN,auto-sd,leve,NaN,Masculino,45,4_ 40 a 49 años


In [392]:
columnas_a_eliminar = ['direccion_normalizada', 'otra_direccion', 'calle', 'altura', 'cruce', 'geocodificacion_CABA', 'vehiculo_victima' ]
df_final_lesiones = df_final_lesiones.drop(columnas_a_eliminar, axis=1)

In [396]:
df_final_lesiones.head()

,id_hecho,n_victimas,fecha,franja_hora,comuna,tipo_calle,pos_x,pos_y,victima,acusado,participantes,gravedad,sexo_victima,edad_victima,RANGO_ETARIO
0,LC-2019-0000179,1,2019-01-01,9,14,NaN,-58.408911,-34.559658,ciclista,NaN,ciclista-sd,leve,NaN,<NA>,NaN
1,LC-2019-0000053,1,2019-01-01,1,8,NaN,-58.443510,-34.669125,auto,NaN,auto-sd,leve,Masculino,57,5_ 50 a 59 años
2,LC-2019-0000063,1,2019-01-01,2,8,NaN,-58.468335,-34.677556,NaN,NaN,sd-sd,leve,NaN,<NA>,NaN
3,LC-2019-0000079,1,2019-01-01,2,7,NaN,-58.437425,-34.647349,peatón,NaN,peaton-sd,leve,Masculino,<NA>,NaN
4,LC-2019-0000082,4,2019-01-01,4,3,NaN,-58.398225,-34.604579,auto,NaN,auto-sd,leve,Masculino,45,4_ 40 a 49 años


## Guardar la data procesada en formato CSV

In [398]:
df_final_lesiones.to_csv('../datasets/lesiones_hechos_victimas.csv', index=False)